# BiLSTM BCE

## Variables d'environnement

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
# Id des GPU disponibles : 0 et 1

## Importation

In [2]:
import os
import time
import datetime
from typing import Any, Union, Dict, List
import uuid
import json
import functools

import pandas as pd
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchtext
import nltk
import sklearn
import transformers
import torchmetrics as tm
from torchmetrics import MetricCollection, Metric, Accuracy, Precision, Recall, AUROC, HammingDistance, F1Score, ROC, AUC, PrecisionRecallCurve


from loguru import logger
from tqdm.auto import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")

## Constantes

In [3]:
CUSTOME_NAME = "glove-bilstm-bce"

# Dataset
DATA_DIR_PATH = os.path.abspath("../../data")
TRAIN_DATASET_PATH = os.path.join(DATA_DIR_PATH, "jigsaw2019-train.csv")
TEST_DATASET_PATH = os.path.join(DATA_DIR_PATH, "jigsaw2019-test.csv")
LABEL_LIST = ['toxicity', 'obscene', 'sexual_explicit',
            'identity_attack', 'insult', 'threat']
IDENTITY_LIST = ['male', 'female', 'transgender', 'other_gender', 'heterosexual',
                'homosexual_gay_or_lesbian', 'bisexual','other_sexual_orientation',
                'christian', 'jewish', 'muslim', 'hindu','buddhist', 'atheist',
                'other_religion', 'black', 'white', 'asian', 'latino',
                'other_race_or_ethnicity', 'physical_disability',
                'intellectual_or_learning_disability',
                'psychiatric_or_mental_illness','other_disability']
SELECTED_IDENTITY_LIST = ['male', 'female', 'black', 'white', 'homosexual_gay_or_lesbian',
                    'christian', 'jewish', 'muslim', 'psychiatric_or_mental_illness']

# Session
SESSION_DIR_PATH = os.path.abspath("../../session")
SESSION_DATETIME = datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S-%f")
SESSION_NAME = f"{CUSTOME_NAME}_{SESSION_DATETIME}"
CURRENT_SESSION_DIR_PATH = os.path.join(SESSION_DIR_PATH, SESSION_NAME)
# Créer le dossier de la session
os.makedirs(CURRENT_SESSION_DIR_PATH, exist_ok=True)

# Architecture de fichier dans `CURRENT_SESSION_DIR_PATH`
LOG_FILE_NAME = f"{SESSION_NAME}.loguru.log"
MODEL_FILE_NAME = f"{SESSION_NAME}.state_dict.model"
TEST_FILE_NAME = f"{SESSION_NAME}.test.csv"
VALIDATION_DATASET_NAME = f"{SESSION_NAME}.jigsaw2019-validation.csv"
VALIDATION_FILE_NAME = f"{SESSION_NAME}.validation.csv"
METRIC_FILE_NAME = f"{SESSION_NAME}.metric.json"
LOG_FILE_PATH = os.path.join(CURRENT_SESSION_DIR_PATH, LOG_FILE_NAME)
MODEL_FILE_PATH = os.path.join(CURRENT_SESSION_DIR_PATH, MODEL_FILE_NAME)
TEST_FILE_PATH = os.path.join(CURRENT_SESSION_DIR_PATH, TEST_FILE_NAME)
VALIDATION_DATASET_FILE_PATH = os.path.join(CURRENT_SESSION_DIR_PATH, VALIDATION_DATASET_NAME)
VALIDATION_FILE_PATH = os.path.join(CURRENT_SESSION_DIR_PATH, VALIDATION_FILE_NAME)
METRIC_FILE_PATH = os.path.join(CURRENT_SESSION_DIR_PATH, METRIC_FILE_NAME)

# CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Logging

In [4]:
logger.add(LOG_FILE_PATH, level="TRACE")
logger.info(f"{SESSION_NAME=}")
logger.info(f"{TRAIN_DATASET_PATH=}")
logger.info(f"{TEST_DATASET_PATH=}")
logger.info(f"{CURRENT_SESSION_DIR_PATH=}")
logger.info(f"{LABEL_LIST=}")
logger.info(f"{IDENTITY_LIST=}")
logger.info(f"{SELECTED_IDENTITY_LIST=}")

2022-03-31 18:54:20.628 | INFO     | __main__:<cell line: 2>:2 - SESSION_NAME='glove-bilstm-bce_2022-03-31T18-54-19-824060'
2022-03-31 18:54:20.629 | INFO     | __main__:<cell line: 3>:3 - TRAIN_DATASET_PATH='/work2/home/ing1/corentin/hatespeech-detection-models/data/jigsaw2019-train.csv'
2022-03-31 18:54:20.630 | INFO     | __main__:<cell line: 4>:4 - TEST_DATASET_PATH='/work2/home/ing1/corentin/hatespeech-detection-models/data/jigsaw2019-test.csv'
2022-03-31 18:54:20.631 | INFO     | __main__:<cell line: 5>:5 - CURRENT_SESSION_DIR_PATH='/work2/home/ing1/corentin/hatespeech-detection-models/session/glove-bilstm-bce_2022-03-31T18-54-19-824060'
2022-03-31 18:54:20.632 | INFO     | __main__:<cell line: 6>:6 - LABEL_LIST=['toxicity', 'obscene', 'sexual_explicit', 'identity_attack', 'insult', 'threat']
2022-03-31 18:54:20.633 | INFO     | __main__:<cell line: 7>:7 - IDENTITY_LIST=['male', 'female', 'transgender', 'other_gender', 'heterosexual', 'homosexual_gay_or_lesbian', 'bisexual', 'oth

## Vérifier la cohérence de l'architecture et l'accès aux ressources

In [5]:
logger.info(f"Checking consistency...")

# Vérifier l'accès aux datasets
if not os.path.exists(TRAIN_DATASET_PATH):
    logger.critical(f"Train dataset does not exist !")
    raise RuntimeError("Train dataset does not exist !")
if not os.path.exists(TEST_DATASET_PATH):
    logger.critical(f"Test dataset does not exist !")
    raise RuntimeError("Test dataset does not exist !")
logger.success("Datasets are reachable")

# Vérifier l'accès aux GPU
GPU_IS_AVAILABLE = torch.cuda.is_available()
GPU_COUNT = torch.cuda.device_count()
logger.info(f"{GPU_IS_AVAILABLE=}")
logger.info(f"{GPU_COUNT=}")
if not GPU_IS_AVAILABLE:
    logger.critical("GPU and CUDA are not available !")
    raise RuntimeError("GPU and CUDA are not available !")
logger.success("GPU and CUDA are available")
logger.info(f"{device=}")
for gpu_id in range(GPU_COUNT):
    gpu_name = torch.cuda.get_device_name(0)
    logger.info(f"GPU {gpu_id} : {gpu_name}")

2022-03-31 18:54:21.006 | INFO     | __main__:<cell line: 1>:1 - Checking consistency...
2022-03-31 18:54:21.008 | SUCCESS  | __main__:<cell line: 10>:10 - Datasets are reachable
2022-03-31 18:54:21.008 | INFO     | __main__:<cell line: 15>:15 - GPU_IS_AVAILABLE=True
2022-03-31 18:54:21.009 | INFO     | __main__:<cell line: 16>:16 - GPU_COUNT=1
2022-03-31 18:54:21.012 | SUCCESS  | __main__:<cell line: 20>:20 - GPU and CUDA are available
2022-03-31 18:54:21.013 | INFO     | __main__:<cell line: 21>:21 - device=device(type='cuda')
2022-03-31 18:54:21.016 | INFO     | __main__:<cell line: 22>:24 - GPU 0 : NVIDIA TITAN X (Pascal)


## Dataset

In [6]:
all_train_df = pd.read_csv(TRAIN_DATASET_PATH, index_col=0)
logger.success("Dataset loaded !")

2022-03-31 18:54:28.387 | SUCCESS  | __main__:<cell line: 2>:2 - Dataset loaded !


In [7]:
# Pour réduire le nombre d'exemple et savoir sur quel groupes d'identités
# le modèle est entrainé, on prend un sous ensemble du jeu de données
train_df = all_train_df[~all_train_df.white.isna()]
if CUSTOME_NAME.startswith("test"):
    # Si c'est juste une session pour tester le notebook
    logger.debug("Mode test is enabled. The training set has been truncated to 20 000 samples.")
    train_df = train_df[:20_000]
# Pour le jeu de validation, on veut juste avoir des indications de perf
# Peu importe, c'est juste pour voir si l'entraînement s'est bien passé
# Sans pour autant regarder les biais
validation_df = all_train_df[all_train_df.white.isna()].sample(n=10_000)

In [8]:
# Remplacer toutes les colonnes correspondantes aux labels par 1 ou 0
# si la probabilité est supérieure ou égale à 0.5 ou non
train_df[LABEL_LIST] = (train_df[LABEL_LIST]>=0.5).astype(int)
validation_df[LABEL_LIST] = (validation_df[LABEL_LIST]>=0.5).astype(int)

In [9]:
# For GloVe tokenizer and vocabulary
# Get the tokenizer
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
max_len = 70
logger.info(f"Tokenizer : basic_english")
# Build vocab
min_freq = 5
logger.info(f"Tokenizer : {min_freq=}")
special_tokens = ['<unk>', '<pad>']
logger.info(f"Tokenizer : {special_tokens=}")

# Create tokenized column
train_df['tokens'] = train_df['comment_text'].apply(lambda x: tokenizer(x)[:max_len])
logger.info(f"Tokens are stored in column of train_df")

vocab = torchtext.vocab.build_vocab_from_iterator(train_df['tokens'],
                                                  min_freq=min_freq,
                                                  specials=special_tokens)
logger.info(f"Vocab is built")

unk_index = vocab['<unk>']
pad_index = vocab['<pad>']
vocab.set_default_index(unk_index)

2022-03-31 18:54:29.626 | INFO     | __main__:<cell line: 5>:5 - Tokenizer : basic_english
2022-03-31 18:54:29.629 | INFO     | __main__:<cell line: 8>:8 - Tokenizer : min_freq=5
2022-03-31 18:54:29.630 | INFO     | __main__:<cell line: 10>:10 - Tokenizer : special_tokens=['<unk>', '<pad>']
2022-03-31 18:54:37.343 | INFO     | __main__:<cell line: 14>:14 - Tokens are stored in column of train_df
2022-03-31 18:54:39.845 | INFO     | __main__:<cell line: 19>:19 - Vocab is built


In [10]:
class JigsawDataset(Dataset):
    def __init__(self, data_df, tokenizer, vocab):
        self.data = data_df
        self.tokenizer = tokenizer
        self.vocab = vocab

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        comment = self.data.iloc[index]["comment_text"]
        label = torch.tensor(self.data.iloc[index][LABEL_LIST].tolist(), dtype=torch.float)
        
        # Tokenize the sentence
        tokenized_sentence = self.tokenizer(comment)

        # Get ids from tokens
        ids = torch.tensor([self.vocab[token] for token in tokenized_sentence])
        length = len(ids)

        return dict(index=index, ids=ids, length=length, labels=label)

In [11]:
def collate(batch, pad_index):#TODO
    batch_ids = [i['ids'] for i in batch]
    # 'batch_ids': list of tensors
    # each tensor contains the ids for each word

    batch_ids = nn.utils.rnn.pad_sequence(batch_ids, padding_value=pad_index, batch_first=True)
    # 'batch_ids': tensor of shape=(nb_sentences, padded_len)

    #print(">> ", batch)
    batch_length = [i['length'] for i in batch]
    # batch_length: list of tensors (each tensor contains the length of the sequence)
    # len(batch_length): batch_size

    #print(">> ", batch_length)
    batch_length = torch.Tensor(batch_length)
    # batch_length: tensor of shape=(batch_size,)

    batch_label = [i['labels'].numpy() for i in batch]
    # 'batch_label': list of tensor (each tensor contains the label)

    batch_label = torch.Tensor(batch_label)
    # 'batch_label': tensor with all the labels of the batch

    batch_index = [i['index'] for i in batch]
    # 'batch_label': list of tensor (each tensor contains the label)

    batch_index = torch.Tensor(batch_index)
    # 'batch_label': tensor with all the labels of the batch

    batch = {
        'index' : batch_index,
        'ids': batch_ids,
        'labels': batch_label,
        'length': batch_length
    }
    return batch

In [12]:
collate = functools.partial(collate, pad_index=pad_index)

## Model

In [13]:
class BiLSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, n_layers, pad_index, dropout_rate):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_index)
        self.biLSTM = nn.LSTM(embedding_dim, hidden_dim, n_layers, bidirectional=True, dropout=dropout_rate, batch_first=True)

        # 2 * hidden_dim -> because it's Bidirectional LSTM (Outputs are concatenated)
        self.linear = nn.Linear(2 * hidden_dim, len(LABEL_LIST))
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, ids, lengths):
        # ids = [batch_size, padded_seq_len]
        # lenghts = [batch_size]

        embedded = self.embedding(ids)
        # embedded = [batch_size, padded_seq_len, embedding_dim]
        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True,
                                                            enforce_sorted=False)
        
        packed_output, (hidden, cell) = self.biLSTM(packed_embedded)
        # hidden = [n_layers * 2, batch_size, hidden_dim]
        # cell = [n_layers * 2, batch_size, hidden_dim]

        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output, batch_first=True)
        # output = [batch_size, seq_len, hidden_dim * 2]

        hidden = self.dropout(torch.cat([hidden[-1], hidden[-2]], dim=-1))
        # hidden = [batch_size, hidden_dim * 2]

        output = self.linear(hidden)
        # output = [batch_size, output_dim]

        return output

### Instancier le modèle

In [14]:
embedding_dim = 300
hidden_dim = 200
n_layers = 3
dropout = 0.4

model = BiLSTMClassifier(len(vocab), 
                        embedding_dim=embedding_dim,
                        hidden_dim=hidden_dim,
                        n_layers=n_layers,
                        pad_index=pad_index,
                        dropout_rate=dropout)

# vectors = torchtext.vocab.FastText()
vectors = torchtext.vocab.GloVe()

pretrained_embedding = vectors.get_vecs_by_tokens(vocab.get_itos())
# 'pretrained_embedding' shape=(vocab_size, 1, 100)
pretrained_embedding = torch.squeeze(pretrained_embedding)

# set the pretrained embedding
model.embedding.weight.data = pretrained_embedding

## Hyperparamètre

In [15]:
BATCH_SIZE = 32
LR=1e-4
PIN_MEMORY = True
NUM_WORKERS = 0
PREFETCH_FACTOR = 2
NUM_EPOCHS = 1
logger.info(f"{BATCH_SIZE=}")
logger.info(f"{LR=}")
logger.info(f"{PIN_MEMORY=}")
logger.info(f"{NUM_WORKERS=}")
logger.info(f"{PREFETCH_FACTOR=}")
logger.info(f"{NUM_EPOCHS=}")

2022-03-31 18:54:44.441 | INFO     | __main__:<cell line: 7>:7 - BATCH_SIZE=32
2022-03-31 18:54:44.444 | INFO     | __main__:<cell line: 8>:8 - LR=0.0001
2022-03-31 18:54:44.445 | INFO     | __main__:<cell line: 9>:9 - PIN_MEMORY=True
2022-03-31 18:54:44.447 | INFO     | __main__:<cell line: 10>:10 - NUM_WORKERS=0
2022-03-31 18:54:44.449 | INFO     | __main__:<cell line: 11>:11 - PREFETCH_FACTOR=2
2022-03-31 18:54:44.450 | INFO     | __main__:<cell line: 12>:12 - NUM_EPOCHS=1


### Loss

In [16]:
class FocalLoss(nn.Module):
    def __init__(self,
                 gamma: float = 2,
                 reduction: str = "mean",
                 pos_weight: torch.Tensor = None):
        super(FocalLoss, self).__init__()
        self.gamma= gamma
        self.reduction = reduction
        self.pos_weight = pos_weight

    def forward(self, inputs: torch.Tensor,
                targets: torch.Tensor):
        p = torch.sigmoid(inputs)
        ce_loss = F.binary_cross_entropy_with_logits(
            inputs, targets, reduction="none", pos_weight=self.pos_weight
        )
        p_t =  p * targets + (1 - p) * (1 - targets)
        loss = ce_loss * ((1 - p_t) ** self.gamma)

        if self.reduction == "mean":
            loss = loss.mean()
        elif self.reduction == "sum":
            loss = loss.sum()

        return loss

In [17]:
# Taken from: https://github.com/Roche/BalancedLossNLP

class ResampleLoss(nn.Module):

    def __init__(self,
                 use_sigmoid=True, partial=False,
                 loss_weight=1.0, reduction='mean',
                 reweight_func=None,  # None, 'inv', 'sqrt_inv', 'rebalance', 'CB'
                 weight_norm=None, # None, 'by_instance', 'by_batch'
                 focal=dict(
                     focal=True,
                     alpha=0.5,
                     gamma=2,
                 ),
                 map_param=dict(
                     alpha=10.0,
                     beta=0.2,
                     gamma=0.1
                 ),
                 CB_loss=dict(
                     CB_beta=0.9,
                     CB_mode='average_w'  # 'by_class', 'average_n', 'average_w', 'min_n'
                 ),
                 logit_reg=dict(
                     neg_scale=5.0,
                     init_bias=0.1
                 ),
                 class_freq=None,
                 train_num=None):
        super(ResampleLoss, self).__init__()

        assert (use_sigmoid is True) or (partial is False)
        self.use_sigmoid = use_sigmoid
        self.partial = partial
        self.loss_weight = loss_weight
        self.reduction = reduction
        if self.use_sigmoid:
            if self.partial:
                raise RuntimeError("Not defined here")
                self.cls_criterion = partial_cross_entropy
            else:
                self.cls_criterion = binary_cross_entropy
        else:
            raise RuntimeError("Not defined here")
            self.cls_criterion = cross_entropy

        # reweighting function
        self.reweight_func = reweight_func

        # normalization (optional)
        self.weight_norm = weight_norm

        # focal loss params
        self.focal = focal['focal']
        self.gamma = focal['gamma']
        self.alpha = focal['alpha'] # change to alpha

        # mapping function params
        self.map_alpha = map_param['alpha']
        self.map_beta = map_param['beta']
        self.map_gamma = map_param['gamma']

        # CB loss params (optional)
        self.CB_beta = CB_loss['CB_beta']
        self.CB_mode = CB_loss['CB_mode']

        self.class_freq = torch.from_numpy(np.asarray(class_freq)).float().cuda()
        self.num_classes = self.class_freq.shape[0]
        self.train_num = train_num # only used to be divided by class_freq
        # regularization params
        self.logit_reg = logit_reg
        self.neg_scale = logit_reg[
            'neg_scale'] if 'neg_scale' in logit_reg else 1.0
        init_bias = logit_reg['init_bias'] if 'init_bias' in logit_reg else 0.0
        # bug fixed https://github.com/wutong16/DistributionBalancedLoss/issues/8
        self.init_bias = - torch.log(
            self.train_num / self.class_freq - 1) * init_bias

        self.freq_inv = torch.ones(self.class_freq.shape).cuda() / self.class_freq
        self.propotion_inv = self.train_num / self.class_freq

    def forward(self,
                cls_score,
                label,
                weight=None,
                avg_factor=None,
                reduction_override=None,
                **kwargs):

        assert reduction_override in (None, 'none', 'mean', 'sum')
        reduction = (
            reduction_override if reduction_override else self.reduction)

        weight = self.reweight_functions(label)

        cls_score, weight = self.logit_reg_functions(label.float(), cls_score, weight)

        if self.focal:
            logpt = self.cls_criterion(
                cls_score.clone(), label, weight=None, reduction='none',
                avg_factor=avg_factor)
            # pt is sigmoid(logit) for pos or sigmoid(-logit) for neg
            pt = torch.exp(-logpt)
            wtloss = self.cls_criterion(
                cls_score, label.float(), weight=weight, reduction='none')
            alpha_t = torch.where(label==1, self.alpha, 1-self.alpha)
            loss = alpha_t * ((1 - pt) ** self.gamma) * wtloss # balance_param should be a tensor
            loss = reduce_loss(loss, reduction)             # add reduction
        else:
            loss = self.cls_criterion(cls_score, label.float(), weight,
                                      reduction=reduction)

        loss = self.loss_weight * loss
        return loss

    def reweight_functions(self, label):
        if self.reweight_func is None:
            return None
        elif self.reweight_func in ['inv', 'sqrt_inv']:
            weight = self.RW_weight(label.float())
        elif self.reweight_func in 'rebalance':
            weight = self.rebalance_weight(label.float())
        elif self.reweight_func in 'CB':
            weight = self.CB_weight(label.float())
        else:
            return None

        if self.weight_norm is not None:
            if 'by_instance' in self.weight_norm:
                max_by_instance, _ = torch.max(weight, dim=-1, keepdim=True)
                weight = weight / max_by_instance
            elif 'by_batch' in self.weight_norm:
                weight = weight / torch.max(weight)

        return weight

    def logit_reg_functions(self, labels, logits, weight=None): 
        if not self.logit_reg:
            return logits, weight
        if 'init_bias' in self.logit_reg:
            logits += self.init_bias
        if 'neg_scale' in self.logit_reg:
            logits = logits * (1 - labels) * self.neg_scale  + logits * labels
            if weight is not None:
                weight = weight / self.neg_scale * (1 - labels) + weight * labels
        return logits, weight

    def rebalance_weight(self, gt_labels):
        repeat_rate = torch.sum( gt_labels.float() * self.freq_inv, dim=1, keepdim=True)
        pos_weight = self.freq_inv.clone().detach().unsqueeze(0) / repeat_rate
        # pos and neg are equally treated
        weight = torch.sigmoid(self.map_beta * (pos_weight - self.map_gamma)) + self.map_alpha
        return weight

    def CB_weight(self, gt_labels):
        if  'by_class' in self.CB_mode:
            weight = torch.tensor((1 - self.CB_beta)).cuda() / \
                     (1 - torch.pow(self.CB_beta, self.class_freq)).cuda()
        elif 'average_n' in self.CB_mode:
            avg_n = torch.sum(gt_labels * self.class_freq, dim=1, keepdim=True) / \
                    torch.sum(gt_labels, dim=1, keepdim=True)
            weight = torch.tensor((1 - self.CB_beta)).cuda() / \
                     (1 - torch.pow(self.CB_beta, avg_n)).cuda()
        elif 'average_w' in self.CB_mode:
            weight_ = torch.tensor((1 - self.CB_beta)).cuda() / \
                      (1 - torch.pow(self.CB_beta, self.class_freq)).cuda()
            weight = torch.sum(gt_labels * weight_, dim=1, keepdim=True) / \
                     torch.sum(gt_labels, dim=1, keepdim=True)
        elif 'min_n' in self.CB_mode:
            min_n, _ = torch.min(gt_labels * self.class_freq +
                                 (1 - gt_labels) * 100000, dim=1, keepdim=True)
            weight = torch.tensor((1 - self.CB_beta)).cuda() / \
                     (1 - torch.pow(self.CB_beta, min_n)).cuda()
        else:
            raise NameError
        return weight

    def RW_weight(self, gt_labels, by_class=True):
        if 'sqrt' in self.reweight_func:
            weight = torch.sqrt(self.propotion_inv)
        else:
            weight = self.propotion_inv
        if not by_class:
            sum_ = torch.sum(weight * gt_labels, dim=1, keepdim=True)
            weight = sum_ / torch.sum(gt_labels, dim=1, keepdim=True)
        return weight
    

def reduce_loss(loss, reduction):
    """Reduce loss as specified.
    Args:
        loss (Tensor): Elementwise loss tensor.
        reduction (str): Options are "none", "mean" and "sum".
    Return:
        Tensor: Reduced loss tensor.
    """
    reduction_enum = F._Reduction.get_enum(reduction)
    # none: 0, elementwise_mean:1, sum: 2
    if reduction_enum == 0:
        return loss
    elif reduction_enum == 1:
        return loss.mean()
    elif reduction_enum == 2:
        return loss.sum()


def weight_reduce_loss(loss, weight=None, reduction='mean', avg_factor=None):
    """Apply element-wise weight and reduce loss.
    Args:
        loss (Tensor): Element-wise loss.
        weight (Tensor): Element-wise weights.
        reduction (str): Same as built-in losses of PyTorch.
        avg_factor (float): Avarage factor when computing the mean of losses.
    Returns:
        Tensor: Processed loss values.
    """
    # if weight is specified, apply element-wise weight
    if weight is not None:
        loss = loss * weight

    # if avg_factor is not specified, just reduce the loss
    if avg_factor is None:
        loss = reduce_loss(loss, reduction)
    else:
        # if reduction is mean, then average the loss by avg_factor
        if reduction == 'mean':
            loss = loss.sum() / avg_factor
        # if reduction is 'none', then do nothing, otherwise raise an error
        elif reduction != 'none':
            raise ValueError('avg_factor can not be used with reduction="sum"')
    return loss


def binary_cross_entropy(pred,
                         label,
                         weight=None,
                         reduction='mean',
                         avg_factor=None):

    # weighted element-wise losses
    if weight is not None:
        weight = weight.float()

    loss = F.binary_cross_entropy_with_logits(
        pred, label.float(), weight, reduction='none')
    loss = weight_reduce_loss(loss, reduction=reduction, avg_factor=avg_factor)

    return loss

In [18]:
def get_label_weights_bce(df, classes=LABEL_LIST):
    weights = torch.empty((len(classes),))

    nb_samples = len(df)

    for idx, c in enumerate(classes):
        nb_zeros = len(df[df[c] == 0])
        nb_ones = nb_samples - nb_zeros
        weights[idx] = nb_zeros / nb_ones

    return weights

def get_label_inv_freq(df, classes=LABEL_LIST):
    weights = torch.empty((len(classes),))
    nb_samples = len(df)

    for idx, c in enumerate(classes):
        nb_zeros = len(df[df[c] == 0])
        weights[idx] = (nb_zeros / nb_samples)

    return weights

def get_nb_samples_lab(df, classes=LABEL_LIST):
    nb_ones_tot, nb_zeros_tot = [], []
    nb_tot = len(df)

    for c in classes:
        nb_zeros = len(df[df[c] == 0])
        nb_ones = nb_tot - nb_zeros

        nb_ones_tot.append(nb_ones)
        nb_zeros_tot.append(nb_zeros)

    return torch.tensor(nb_ones_tot), torch.tensor(nb_zeros_tot)

### Instancier les différents objets

In [19]:
train_dataset = JigsawDataset(train_df, tokenizer,
                              vocab=vocab)
train_dataloader = DataLoader(train_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=True,
                             num_workers=NUM_WORKERS,
                             prefetch_factor=PREFETCH_FACTOR,
                             pin_memory=PIN_MEMORY,
                             collate_fn=collate)

validation_dataset = JigsawDataset(validation_df, tokenizer,
                              vocab=vocab)
validation_dataloader = DataLoader(validation_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=True,
                             num_workers=NUM_WORKERS,
                             prefetch_factor=PREFETCH_FACTOR,
                             pin_memory=PIN_MEMORY,
                             collate_fn=collate)

# Pas besoin de Sigmoid en sorti du model seulement pour `BCEWithLogitsLoss`
criterion = torch.nn.BCEWithLogitsLoss()
logger.info(criterion)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
logger.info(optimizer)

model.to(device)
criterion.to(device)

2022-03-31 18:54:45.478 | INFO     | __main__:<cell line: 23>:23 - BCEWithLogitsLoss()
2022-03-31 18:54:45.480 | INFO     | __main__:<cell line: 25>:25 - AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    maximize: False
    weight_decay: 0.01
)


BCEWithLogitsLoss()

## Metric

### Variantes de Hamming Loss

In [20]:
class HammingLossWithoutThreshold(Metric):
    def __init__(self, num_classes=1, dist_sync_on_step=False):
        super().__init__(dist_sync_on_step=dist_sync_on_step)
        self.num_classes = num_classes

        self.add_state("total", default=torch.tensor(0, dtype=torch.float32), dist_reduce_fx="sum")
        self.add_state("nbr_sample", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        current_nbr_sample, current_nbr_category = preds.shape
        if current_nbr_category != self.num_classes:
          raise AttributeError("`num_classes` != `current_nbr_category` detected in `pred` parameter")
        
        current_loss_per_pred = torch.absolute(target - preds)
        current_hamming_loss = current_loss_per_pred.sum()

        self.total += current_hamming_loss.float()
        self.nbr_sample += current_nbr_sample

    def compute(self):
        return self.total/(self.num_classes*self.nbr_sample)

In [21]:
class RebalancedHammingLossWithoutThreshold(Metric):
    def __init__(self, num_classes=1, average="macro", dist_sync_on_step=False):
        super().__init__(dist_sync_on_step=dist_sync_on_step)
        self.num_classes = num_classes

        # average = "macro" or None
        self.average = average

        # Nombre de positif 1 & negatif 0 par categorie
        self.add_state(
            "number_positive",
            default=torch.tensor([0 for _ in range(num_classes)]),
            dist_reduce_fx="sum",
        )
        self.add_state(
            "number_negative",
            default=torch.tensor([0 for _ in range(num_classes)]),
            dist_reduce_fx="sum",
        )

        self.add_state(
            "hamming_loss_positive",
            default=torch.tensor([0.0 for _ in range(num_classes)]),
            dist_reduce_fx="sum",
        )
        self.add_state(
            "hamming_loss_negative",
            default=torch.tensor([0.0 for _ in range(num_classes)]),
            dist_reduce_fx="sum",
        )

        self.add_state("nbr_sample", default=torch.tensor(0), dist_reduce_fx="sum")

    def update(self, preds: torch.Tensor, target: torch.Tensor):
        current_nbr_sample, current_nbr_category = preds.shape
        if current_nbr_category != self.num_classes:
            raise AttributeError(
                "`num_classes` != `current_nbr_category` detected in `pred` parameter"
            )

        # Nombre de positif 1 & negatif 0 par categorie
        current_number_positive = target.sum(axis=0)
        current_number_negative = current_nbr_sample - target.sum(axis=0)

        self.number_positive += current_number_positive.int()
        self.number_negative += current_number_negative.int()

        self.nbr_sample += current_nbr_sample

        for class_id in range(self.num_classes):
            positive_filter = target[:, class_id] == 1
            negative_filter = target[:, class_id] == 0

            target_vector = target[:, class_id]
            preds_vector = preds[:, class_id]

            # Filtered vector
            ## Target
            pos_filtered_target_vector = target_vector[positive_filter]
            neg_filtered_target_vector = target_vector[negative_filter]
            ## Preds
            pos_filtered_preds_vector = preds_vector[positive_filter]
            neg_filtered_preds_vector = preds_vector[negative_filter]

            # Hamming Loss without Threshold
            hamming_loss_on_positive = torch.absolute(
                pos_filtered_target_vector - pos_filtered_preds_vector
            )
            hamming_loss_on_negative = torch.absolute(
                neg_filtered_target_vector - neg_filtered_preds_vector
            )

            self.hamming_loss_positive[class_id] += hamming_loss_on_positive.sum()
            self.hamming_loss_negative[class_id] += hamming_loss_on_negative.sum()

    def compute(self):
        factor_pos = self.nbr_sample / (2 * self.number_positive)
        factor_neg = self.nbr_sample / (2 * self.number_negative)

        rebalanced_hamming_loss_per_class = torch.multiply(
            self.hamming_loss_positive, factor_pos
        ) + torch.multiply(self.hamming_loss_negative, factor_neg)
        if self.average == "macro":
            return rebalanced_hamming_loss_per_class.sum() / (
                self.nbr_sample * self.num_classes
            )
        return rebalanced_hamming_loss_per_class / (self.nbr_sample)


### Instanciation des metrics

In [22]:
num_classes = len(LABEL_LIST)
train_metric_dict = dict()

# AUROC Macro
auroc_macro = AUROC(num_classes=num_classes, compute_on_step=True, average="macro")
train_metric_dict["auroc_macro"] = auroc_macro

# AUROC per class
auroc_per_class = AUROC(num_classes=num_classes, compute_on_step=True, average=None)
train_metric_dict["auroc_per_class"] = auroc_per_class

# F1 score global
f1 = F1Score()
train_metric_dict["f1"] = f1

# F1 score per class
f1_per_calss = F1Score(num_classes=6, average=None)
train_metric_dict["f1_per_calss"] = f1_per_calss

# Hamming Distance without Threshold
hamming_loss_woutt = HammingLossWithoutThreshold(num_classes=num_classes)
train_metric_dict["hamming_loss_without_threshold"] = hamming_loss_woutt

# Rebalanced Hamming Distance without Threshold macro
rebalanced_hamming_loss_woutt_macro = RebalancedHammingLossWithoutThreshold(
    num_classes=num_classes, average="macro"
)
train_metric_dict[
    "rebalanced_hamming_loss_without_threshold_macro"
] = rebalanced_hamming_loss_woutt_macro

# Rebalanced Hamming Distance without Threshold macro
rebalanced_hamming_loss_woutt_per_class = RebalancedHammingLossWithoutThreshold(
    num_classes=num_classes, average=None
)
train_metric_dict[
    "rebalanced_hamming_loss_without_threshold_per_class"
] = rebalanced_hamming_loss_woutt_per_class

In [23]:
train_metric = MetricCollection(train_metric_dict)
train_metric.to(device)

validation_metric = train_metric.clone()
validation_metric.to(device)

MetricCollection(
  (auroc_macro): AUROC()
  (auroc_per_class): AUROC()
  (f1): F1Score()
  (f1_per_calss): F1Score()
  (hamming_loss_without_threshold): HammingLossWithoutThreshold()
  (rebalanced_hamming_loss_without_threshold_macro): RebalancedHammingLossWithoutThreshold()
  (rebalanced_hamming_loss_without_threshold_per_class): RebalancedHammingLossWithoutThreshold()
)

### Export metrics

In [24]:
def serialize(object_to_serialize: Any, ensure_ascii: bool = True) -> str:
    """
    Serialize any object, i.e. convert an object to JSON
    Args:
        object_to_serialize (Any): The object to serialize
        ensure_ascii (bool, optional): If ensure_ascii is true (the default), the output is guaranteed to have all incoming non-ASCII characters escaped. If ensure_ascii is false, these characters will be output as-is. Defaults to True.
    Returns:
            str: string of serialized object (JSON)
    """

    def dumper(obj: Any) -> Union[str, Dict]:
        """
        Function called recursively by json.dumps to know how to serialize an object.
        For example, for datetime, we try to convert it to ISO format rather than
        retrieve the list of attributes defined in its object.
        Args:
            obj (Any): The object to serialize
        Returns:
            Union[str, Dict]: Serialized object
        """
        if isinstance(obj, torch.Tensor):
            return obj.cpu().numpy().tolist()
        elif hasattr(obj, "__dict__"):
            return obj.__dict__
        return str(obj)

    return json.dumps(object_to_serialize, default=dumper, ensure_ascii=ensure_ascii)

In [25]:
def export_metric(metric_collection, **kwargs):
    """
    Export MetricCollection to json file

    Args:
        metric_collection: MetricCollection
        **kwargs: field to add in json line
    """
    with open(METRIC_FILE_PATH, "a") as f:
        metric_collection_value = metric_collection.compute()
        metric_collection_value.update(kwargs)
        serialized_value = serialize(metric_collection_value)
        f.write(serialized_value)
        f.write("\n")
    logger.success("Metrics are exported !")

## Entraînement

In [26]:
# progress = tqdm(train_dataloader, desc="valid batch...", leave=False)
# for _, batch in enumerate(progress):
    
#     token_list_batch = batch["ids"].to(device)
#     label_batch = batch["labels"].to(device)
#     length_batch = batch["length"].to(device)

#     # Predict
#     prediction_batch = model(token_list_batch, length_batch)
#     print(prediction_batch.shape)
#     break

In [27]:
def train_epoch(epoch_id=None):
    model.train()
    logger.info(f"START EPOCH {epoch_id=}")

    progress = tqdm(train_dataloader, desc='training batch...', leave=False)
    for batch_id, batch in enumerate(progress):
        if batch_id % 1_000 == 0:
            valid_epoch(epoch_id=epoch, batch_id=batch_id)
            model.train()
        
        logger.trace(f"{batch_id=}")
        token_list_batch = batch["ids"].to(device)
        label_batch = batch["labels"].to(device)
        length_batch = batch["length"].to(device)

        # Reset gradient
        optimizer.zero_grad()

        # Predict
        prediction_batch = model(token_list_batch, length_batch)
        transformed_prediction_batch = prediction_batch.squeeze()

        # Loss
        loss = criterion(transformed_prediction_batch.to(torch.float32), label_batch.to(torch.float32))

        # Metrics
        proba_prediction_batch = torch.sigmoid(transformed_prediction_batch)
        train_metrics_collection_dict = train_metric(proba_prediction_batch.to(torch.float32), label_batch.to(torch.int32))
        logger.trace(train_metrics_collection_dict)

        # Backprop        
        loss.backward()
        # gradient clip
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        # Update progress bar description
        progress_description = "Train Loss : {loss:.4f} - Train AUROC : {acc:.4f}"
        auroc_macro_value = float(train_metrics_collection_dict["auroc_macro"])
        progress_description = progress_description.format(loss=loss.item(), acc=auroc_macro_value)
        progress.set_description(progress_description)

    logger.info(f"END EPOCH {epoch_id=}")

In [28]:
@torch.no_grad()
def valid_epoch(epoch_id=None, batch_id=None):
    model.eval()
    logger.info(f"START VALIDATION {epoch_id=}{batch_id=}")
    validation_metric.reset()

    loss_list = []
    prediction_list = torch.Tensor([])
    target_list = torch.Tensor([])


    progress = tqdm(validation_dataloader, desc="valid batch...", leave=False)
    for _, batch in enumerate(progress):
        
        token_list_batch = batch["ids"].to(device)
        label_batch = batch["labels"].to(device)
        length_batch = batch["length"].to(device)

        # Predict
        prediction_batch = model(token_list_batch, length_batch)

        transformed_prediction_batch = prediction_batch.squeeze()

        # Loss
        loss = criterion(
            transformed_prediction_batch.to(torch.float32),
            label_batch.to(torch.float32),
        )

        loss_list.append(loss.item())

        proba_prediction_batch = torch.sigmoid(transformed_prediction_batch)
        prediction_list = torch.concat(
            [prediction_list, proba_prediction_batch.cpu()]
        )
        target_list = torch.concat([target_list, label_batch.cpu()])

        # Metrics
        validation_metric(proba_prediction_batch.to(torch.float32), label_batch.to(torch.int32))

    loss_mean = np.mean(loss_list)
    logger.trace(validation_metric.compute())
    logger.info(f"END VALIDATION {epoch_id=}{batch_id=}")
    export_metric(validation_metric, epoch_id=epoch_id, batch_id=batch_id, loss=loss_mean)

In [29]:
torch.cuda.empty_cache()
progress =  tqdm(range(1,NUM_EPOCHS+1), desc='training epoch...', leave=True)
for epoch in progress:
    # Train
    train_epoch(epoch_id=epoch)

    # Validation
    valid_epoch(epoch_id=epoch)

    # Save
    # torch.save(model, MODEL_FILE_PATH)
    torch.save(model.state_dict(), MODEL_FILE_PATH)

training epoch...:   0%|          | 0/1 [00:00<?, ?it/s]

2022-03-31 18:54:49.132 | INFO     | __main__:train_epoch:3 - START EPOCH epoch_id=1


training batch...:   0%|          | 0/12648 [00:00<?, ?it/s]

2022-03-31 18:54:49.241 | INFO     | __main__:valid_epoch:4 - START VALIDATION epoch_id=1batch_id=0


valid batch...:   0%|          | 0/313 [00:00<?, ?it/s]

...

2022-03-31 19:15:36.981 | INFO     | __main__:valid_epoch:43 - END VALIDATION epoch_id=1batch_id=11000
2022-03-31 19:15:36.982 | SUCCESS  | __main__:export_metric:15 - Metrics are exported !
2022-03-31 19:17:10.366 | INFO     | __main__:valid_epoch:4 - START VALIDATION epoch_id=1batch_id=12000


valid batch...:   0%|          | 0/313 [00:00<?, ?it/s]

2022-03-31 19:17:28.449 | INFO     | __main__:valid_epoch:43 - END VALIDATION epoch_id=1batch_id=12000
2022-03-31 19:17:28.450 | SUCCESS  | __main__:export_metric:15 - Metrics are exported !
2022-03-31 19:18:29.174 | INFO     | __main__:train_epoch:43 - END EPOCH epoch_id=1
2022-03-31 19:18:29.175 | INFO     | __main__:valid_epoch:4 - START VALIDATION epoch_id=1batch_id=None


valid batch...:   0%|          | 0/313 [00:00<?, ?it/s]

2022-03-31 19:18:47.379 | INFO     | __main__:valid_epoch:43 - END VALIDATION epoch_id=1batch_id=None
2022-03-31 19:18:47.380 | SUCCESS  | __main__:export_metric:15 - Metrics are exported !


## Evaluation

In [30]:
try:
    del train_df
    del validation_df
except NameError:
    logger.warning("Train DataFrame & Validation DataFrame already deleted")

In [31]:
test_df = pd.read_csv(TEST_DATASET_PATH, index_col=0)

In [32]:
test_df = test_df[~test_df.white.isna()]

In [33]:
test_dataset = JigsawDataset(test_df, tokenizer,
                              vocab=vocab)
test_dataloader = DataLoader(test_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             collate_fn=collate)

In [34]:
@torch.no_grad()
def evaluation(model):
    model.eval()
    logger.info(f"START EVALUATION")

    index_tensor = torch.Tensor([])
    prediction_tensor = torch.Tensor([])

    progress = tqdm(test_dataloader, desc='test batch...', leave=False)
    for batch_id, batch in enumerate(progress):
        logger.trace(f"{batch_id=}")
        index_batch = batch["index"].to(device)
        token_list_batch = batch["ids"].to(device)
        label_batch = batch["labels"].to(device)
        length_batch = batch["length"].to(device)


        # Predict
        prediction_batch = model(token_list_batch, length_batch)
        transformed_prediction_batch = prediction_batch.squeeze()
        proba_prediction_batch = torch.sigmoid(transformed_prediction_batch)
        
        index_tensor = torch.concat([index_tensor, index_batch.cpu()])
        prediction_tensor = torch.concat([prediction_tensor, proba_prediction_batch.cpu()])
    
    logger.info(f"END EVALUATION")
    prediction_test_df = pd.DataFrame(prediction_tensor.tolist(), 
                                     columns=LABEL_LIST,
                                     index=index_tensor.to(int).tolist())
    prediction_test_df.to_csv(TEST_FILE_PATH)
    logger.success(f"Test predictions exported !")

In [35]:
evaluation(model)

2022-03-31 19:18:49.594 | INFO     | __main__:evaluation:4 - START EVALUATION


test batch...:   0%|          | 0/1339 [00:00<?, ?it/s]

2022-03-31 19:19:39.167 | INFO     | __main__:evaluation:26 - END EVALUATION
2022-03-31 19:19:39.613 | SUCCESS  | __main__:evaluation:31 - Test predictions exported !


## Exporter les prédictions de la dataset de validation

In [36]:
@torch.no_grad()
def export_validation(model):
    model.eval()
    logger.info(f"START GET PREDICTION ON VALIDATION DATASET")

    index_tensor = torch.Tensor([])
    prediction_tensor = torch.Tensor([])
    label_tensor = torch.Tensor([])

    progress = tqdm(validation_dataloader, desc='valid batch...', leave=False)
    for batch_id, batch in enumerate(progress):
        logger.trace(f"{batch_id=}")
        index_batch = batch["index"].to(device)
        token_list_batch = batch["ids"].to(device)
        label_batch = batch["labels"].to(device)
        length_batch = batch["length"].to(device)


        # Predict
        prediction_batch = model(token_list_batch, length_batch)
        transformed_prediction_batch = prediction_batch.squeeze()
        proba_prediction_batch = torch.sigmoid(transformed_prediction_batch)
        
        index_tensor = torch.concat([index_tensor, index_batch.cpu()])
        prediction_tensor = torch.concat([prediction_tensor, proba_prediction_batch.cpu()])
        label_tensor = torch.concat([label_tensor, label_batch.cpu()])
    
    logger.info(f"END GET PREDICTION ON VALIDATION DATASET")
    prediction_valid_df = pd.DataFrame(prediction_tensor.tolist(), 
                                     columns=LABEL_LIST,
                                     index=index_tensor.to(int).tolist())
    label_valid_df = pd.DataFrame(label_tensor.tolist(), 
                                     columns=LABEL_LIST,
                                     index=index_tensor.to(int).tolist())
    prediction_valid_df.to_csv(VALIDATION_FILE_PATH)
    label_valid_df.to_csv(VALIDATION_DATASET_FILE_PATH)
    logger.success(f"Validation predictions exported !")
export_validation(model)

2022-03-31 19:19:39.862 | INFO     | __main__:export_validation:4 - START GET PREDICTION ON VALIDATION DATASET


valid batch...:   0%|          | 0/313 [00:00<?, ?it/s]

2022-03-31 19:19:51.402 | INFO     | __main__:export_validation:28 - END GET PREDICTION ON VALIDATION DATASET
2022-03-31 19:19:51.556 | SUCCESS  | __main__:export_validation:37 - Validation predictions exported !
